In [3]:
from mpi4py import MPI
import numpy as np

def calculate_partial_sum(array, rank, size):
    N = len(array)
    chunk_size = N // size
    start = rank * chunk_size
    end = start + chunk_size if rank != size - 1 else N
    partial_sum = np.sum(array[start:end])
    return partial_sum

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        N = 100  # total number of elements
        array = np.arange(N)
    else:
        array = None

    # Scatter the array to all processes
    array = comm.scatter(array, root=0)

    # Calculate partial sum
    partial_sum = calculate_partial_sum(array, rank, size)

    # Gather all partial sums
    partial_sums = comm.gather(partial_sum, root=0)

    # Display intermediate sums calculated at different processors
    if rank == 0:
        print("Intermediate sums calculated at different processors:")
        for i, p_sum in enumerate(partial_sums):
            print(f"Processor {i}: {p_sum}")

        total_sum = np.sum(partial_sums)
        print(f"Total sum: {total_sum}")

if __name__ == "__main__":
    main()


ValueError: expecting 1 items, got 100

In [2]:
!pip install mpi4py


   ---------------------------------------- 0.0/466.1 kB ? eta -:--:--
    --------------------------------------- 10.2/466.1 kB ? eta -:--:--
   ------------- -------------------------- 153.6/466.1 kB 2.3 MB/s eta 0:00:01
   ---------------------------------- ----- 399.4/466.1 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 466.1/466.1 kB 2.9 MB/s eta 0:00:00


In [5]:
from mpi4py import MPI
import numpy as np

def calculate_partial_sum(array_chunk):
    return np.sum(array_chunk)

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        N = 100  # total number of elements
        array = np.arange(N)
        chunk_size = len(array) // size
        # Scatter the array into chunks
        scatter_data = [array[i:i+chunk_size] for i in range(0, len(array), chunk_size)]
    else:
        scatter_data = None

    # Scatter the array chunks to all processes
    local_chunk = comm.scatter(scatter_data, root=0)

    # Calculate partial sum
    partial_sum = calculate_partial_sum(local_chunk)

    # Gather all partial sums
    partial_sums = comm.gather(partial_sum, root=0)

    # Display intermediate sums calculated at different processors
    if rank == 0:
        print("Intermediate sums calculated at different processors:")
        for i, p_sum in enumerate(partial_sums):
            print(f"Processor {i}: {p_sum}")

        total_sum = np.sum(partial_sums)
        print(f"Total sum: {total_sum}")

if __name__ == "__main__":
    main()


Intermediate sums calculated at different processors:
Processor 0: 4950
Total sum: 4950


In [6]:
from mpi4py import MPI
import numpy as np

def calculate_partial_sum(array_chunk):
    return np.sum(array_chunk)

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        N = 100  # total number of elements
        array = np.arange(N)
        chunk_size = len(array) // size
        # Scatter the array into chunks
        scatter_data = [array[i:i+chunk_size] for i in range(0, len(array), chunk_size)]
    else:
        scatter_data = None

    # Scatter the array chunks to all processes
    local_chunk = comm.scatter(scatter_data, root=0)

    # Calculate partial sum
    partial_sum = calculate_partial_sum(local_chunk)

    # Gather all partial sums
    partial_sums = comm.gather((rank, partial_sum), root=0)

    # Display intermediate sums calculated at different processors
    if rank == 0:
        print("Intermediate sums calculated at different processors:")
        for i, (p_rank, p_sum) in enumerate(partial_sums):
            print(f"Processor {p_rank}: {p_sum}")

        total_sum = np.sum([p_sum for _, p_sum in partial_sums])
        print(f"Total sum: {total_sum}")

if __name__ == "__main__":
    main()


Intermediate sums calculated at different processors:
Processor 0: 4950
Total sum: 4950


In [7]:
from mpi4py import MPI
import numpy as np

def calculate_partial_sum(array_chunk, rank):
    partial_sum = np.sum(array_chunk)
    print(f"Processor {rank}: Partial sum calculated = {partial_sum}")
    return partial_sum

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        N = 100  # total number of elements
        array = np.arange(N)
        chunk_size = len(array) // size
        # Scatter the array into chunks
        scatter_data = [array[i:i+chunk_size] for i in range(0, len(array), chunk_size)]
    else:
        scatter_data = None

    # Scatter the array chunks to all processes
    local_chunk = comm.scatter(scatter_data, root=0)

    # Calculate partial sum
    partial_sum = calculate_partial_sum(local_chunk, rank)

    # Gather all partial sums
    partial_sums = comm.gather(partial_sum, root=0)

    # Display intermediate sums calculated at different processors
    if rank == 0:
        print("Intermediate sums calculated at different processors:")
        for i, p_sum in enumerate(partial_sums):
            print(f"Processor {i}: {p_sum}")

        total_sum = np.sum(partial_sums)
        print(f"Total sum: {total_sum}")

if __name__ == "__main__":
    main()


Processor 0: Partial sum calculated = 4950
Intermediate sums calculated at different processors:
Processor 0: 4950
Total sum: 4950


# main code

In [8]:
from mpi4py import MPI
import numpy as np

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    N = 10  # Total number of elements
    array = np.zeros(N, dtype=int)  # Initialize the array
    local_sum = 0  # Partial sum for each processor
    global_sum = 0  # Final sum

    if rank == 0:
        print("Original array:", end=" ")
        for i in range(N):
            array[i] = i + 1
            print(array[i], end=" ")
        print()

    # Scatter the array elements to all processors
    local_array = np.zeros(N // size, dtype=int)
    comm.Scatter(array, local_array, root=0)

    # Calculate local sum
    for i in range(len(local_array)):
        local_sum += local_array[i]

    # Display local sums calculated by each processor
    print(f"Processor {rank} local sum: {local_sum}")

    # Reduce all local sums to get the global sum
    global_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

    # Display the global sum
    if rank == 0:
        print("Global sum:", global_sum)

if __name__ == "__main__":
    main()


Original array: 1 2 3 4 5 6 7 8 9 10 
Processor 0 local sum: 55
Global sum: 55


In [10]:
from mpi4py import MPI
import numpy as np

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    unitsize = 5
    root = 0

    send_buffer = None
    recieve_buffer = np.zeros(unitsize, dtype=int)
    new_recieve_buffer = np.zeros(size, dtype=int)

    # Set data for distribution
    if rank == root:
        total_elements = unitsize * size
        print(f"Enter {total_elements} elements:")
        send_buffer = np.arange(total_elements)
    
    # Scatter data to processes
    comm.Scatter(send_buffer, recieve_buffer, root=root)

    # Calculate sum at non-root processes
    # Store result in first index of array
    recieve_buffer[0] = np.sum(recieve_buffer)

    print(f"Intermediate sum at process {rank} is {recieve_buffer[0]}")

    # Gather data from processes
    comm.Gather(recieve_buffer[0:1], new_recieve_buffer, root=root)

    # Aggregate output from all non-root processes
    if rank == root:
        total_sum = np.sum(new_recieve_buffer)
        print(f"Final sum: {total_sum}")

if __name__ == "__main__":
    main()


Enter 5 elements:
Intermediate sum at process 0 is 10
Final sum: 10
